## Safely Store a Wallet

In [ ]:
#r "nuget:CardanoSharp.Wallet,1.0.0"

Installed Packages CardanoSharp.Wallet, 1.0.0

## Create a Wallet and Derive to Account Node

In [ ]:
using CardanoSharp.Wallet;
using CardanoSharp.Wallet.Enums;
using CardanoSharp.Wallet.Extensions;
using CardanoSharp.Wallet.Extensions.Models;
using CardanoSharp.Wallet.Models.Addresses;
using CardanoSharp.Wallet.Models.Keys;

Mnemonic mnemonic = new MnemonicService().Generate(24);

var accountNode = mnemonic
    .GetMasterNode()
    .Derive(PurposeType.Shelley)
    .Derive(CoinType.Ada)
    .Derive(0);
accountNode.SetPublicKey();

Console.WriteLine($"Account Private Key: {accountNode.PrivateKey.Key.ToStringHex()}");
Console.WriteLine($"Account Public Key: {accountNode.PublicKey.Key.ToStringHex()}");

Account Private Key: 483fb9bffa8731cc44c4992e210398dfe2cb1c849743e9878f213132890c9a59de71b046aa40690920cb5617153adfe8b274375ae6f789f8cf2b6eb9a2d3579b
Account Public Key: 30427c912a75bfb0f68494aa41b380c57a527596917cc6abcf0b1a6557860fb9


## Encrypt Account Node

In [ ]:
using System.Text.Json;

// use the Spending Password to 2-Way encrypt the Private Key
// and then store both the encrypted Private Key and the plain Public Key.
var blob = new Tuple<PrivateKey, PublicKey>(accountNode.PrivateKey.Encrypt("spending_password"), accountNode.PublicKey);
var store = JsonSerializer.Serialize(blob);

Console.WriteLine("Encrypted and Serialized Account Node");
Console.WriteLine(store);

Encrypted and Serialized Account Node
{"Item1":{"Key":"QAAAAC444PDVZ7cJwR2RYjk+WjIQNhzpwCv/903iJ9+xr1M3aNi/MlGqgx9LEPgN9bEi6OrUpDMlvwBkLw2qsbTNbmAFbpC+MW0svog9rgN1kvZmNQv7ABCaTJbK3utnm4LEcQ==","Chaincode":"IAAAAEZXg0GDx9fyKtQk2bEvz3AR8zts5/obcqS3bhawLl9OKOSe1NxnbZdBDmdj9vRJY7LxPB5HL5Dmpr82D8TwXEA="},"Item2":{"Key":"MEJ8kSp1v7D2hJSqQbOAxXpSdZaRfMarzwsaZVeGD7k=","Chaincode":"1vQRomMLrF7ZZJ2xElXQs9TyJbBiT8ybHY4cQbmtT3s="}}


## Decrypt Account Node

In [ ]:
// The user now wants to send a transaction.
// They enter in the ADDR, Amount to Send, and their Spending Password.
// I can now decrypt the Account Private Key...
var decryptedAccountNode = JsonSerializer.Deserialize<Tuple<PrivateKey, PublicKey>>(store);
// This is the decrypted Account
PrivateKey accountPrivate = decryptedAccountNode.Item1.Decrypt("spending_password");
PublicKey accountPublic = decryptedAccountNode.Item2;

Console.WriteLine($"Account Private Key: {accountPrivate.Key.ToStringHex()}");
Console.WriteLine($"Account Public Key: {accountPublic.Key.ToStringHex()}");

Account Private Key: 483fb9bffa8731cc44c4992e210398dfe2cb1c849743e9878f213132890c9a59de71b046aa40690920cb5617153adfe8b274375ae6f789f8cf2b6eb9a2d3579b
Account Public Key: 30427c912a75bfb0f68494aa41b380c57a527596917cc6abcf0b1a6557860fb9
